# 提出用ファイル作成

解の確認と提出用ファイルの作成を行います。

### パッケージ読み込み

In [1]:
from __future__ import annotations
import pandas as pd
import math

### 入力データ読み込み

In [2]:
# データ読み込み
df = pd.read_csv("../data/in/companies.csv", index_col=0)
df.head()

,X,Y,EPT,FPT,PT,PW
CID,,,,,,
1,5,15,118,278,90,20
2,-7,-15,2573,2733,90,10
3,-15,35,2119,2279,90,20
4,-17,2,209,369,90,10
5,25,10,1546,1706,90,30


### 定数の定義

In [3]:
# 定数
# 会社リスト
COMPANIES = df.index.tolist()
# 会社数
COMPANY_CNT = len(COMPANIES)
# 営業リスト
SALES = [1, 2, 3]
# 営業の人数
SALES_CNT = len(SALES)
# 営業のカバン容量
SALES_CAPACITY = 700

### 評価関数値を取得する関数を定義

In [4]:
def get_obj(res: dict[int, list[int]]) -> float:
    """
    解の評価関数値を取得する.

    Args:
        res (dict[int, list[int]]): 解

    Returns:
        float: 評価関数値
    """
    obj = 0
    for k in SALES:
        t = 0
        penalty = 0
        spw = 0
        # スタート地点（原点）
        prev_x, prev_y = 0, 0
        for i in res[k]:
            # 業務時間
            x, y = df.at[i, "X"], df.at[i, "Y"]
            diff_x, diff_y = abs(x - prev_x), abs(y - prev_y)
            t += math.sqrt(diff_x*diff_x + diff_y*diff_y)

            # プレゼン時間に関するペナルティ
            ept, fpt, pt = df.at[i, "EPT"], df.at[i, "FPT"], df.at[i, "PT"]
            penalty += max(ept-t, 0)
            penalty += 2*max(0, (t+pt)-fpt)

            # プレゼン時間を追加
            t += pt

            # 訪問する会社のプレゼン資料の総重量
            pw = df.at[i, "PW"]
            spw += pw

            prev_x, prev_y = x, y
        # 最後に訪問する会社からゴール地点（原点）への業務時間
        x, y = 0, 0
        diff_x, diff_y = abs(x - prev_x), abs(y - prev_y)
        t += math.sqrt(diff_x*diff_x + diff_y*diff_y)

        # カバン容量に関するペナルティ
        penalty += 10*max(0, spw-SALES_CAPACITY)

        obj += t + penalty
    return obj

### 解の作成

結果ファイルより解を作成します。  
**"RESULT_FILE_NAME"を各自が作成した求解結果に変更してください。**

In [5]:
# 結果ファイル読み込み
RESULT_FILE_NAME = "../data/out/sample_res.csv"
df_res = pd.read_csv(RESULT_FILE_NAME)

# カラムチェック
if df_res.columns.tolist() != ["SID", "CID"]:
    print("カラムが異なります。")

df_res.head()

,SID,CID
0,1,80
1,1,4
2,1,92
3,1,45
4,1,17


In [6]:
# 解作成
res = {k: [] for k in SALES}
for i in range(COMPANY_CNT):
    sid = df_res.at[i, "SID"]
    cid = df_res.at[i, "CID"]
    res[sid].append(cid)
obj = get_obj(res)

# 結果出力
for k in SALES:
    print(f"{k}: {res[k]}")
print(obj)

1: [80, 4, 92, 45, 17, 13, 53, 43, 100, 76, 55, 62, 81, 58, 98, 84, 99, 28, 90, 20, 72, 77, 66, 48, 32, 87, 71, 36, 96, 41, 22, 39, 10, 89]
2: [1, 63, 69, 11, 74, 86, 16, 68, 82, 94, 93, 35, 12, 61, 75, 65, 73, 8, 83, 88, 56, 50, 64, 40, 23, 91, 2, 21, 26, 44, 34, 78, 37]
3: [59, 46, 57, 38, 7, 30, 9, 31, 27, 5, 95, 49, 25, 42, 24, 19, 47, 54, 85, 52, 70, 97, 67, 51, 18, 29, 3, 6, 15, 33, 14, 79, 60]
97947.1263979882


### 解の確認

In [7]:
# 結果チェック
# 営業の存在確認
for k in res.keys():
    if not k in SALES:
        print(f"営業{k}は存在しません。")
# 会社の存在確認
for v in res.values():
    for i in v:
        if not i in COMPANIES:
            print(f"会社{i}は存在しません。")
# 会社の訪問数確認
company_cnt = sum(len(v) for v in res.values())
if company_cnt != COMPANY_CNT:
    print("会社の訪問数が会社数と一致しません。")
# 会社の訪問過不足確認
for i in COMPANIES:
    has_company = False
    for v in res.values():
        if i in v:
            has_company = True
    if not has_company:
        print(f"会社{i}を訪問していません。")

### 提出用ファイルの作成

提出用ファイルフォーマットは、「評価関数値_名前.csv」とする。  
**"USER_NAME"を各自の名前に変更してください。**

In [8]:
# 提出用ファイル作成
USER_NAME = "username"
l = []
for k in SALES:
    for i in res[k]:
        l.append([k, i])
sub = pd.DataFrame(l, columns=("SID", "CID"))
# 小数点回避のため、1000倍（小数点以下3桁）した値をファイル名に使用
sub.to_csv(f"../data/out/{int(1000*obj)}_{USER_NAME}.csv", index=None)